In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [2]:
events = utils.get_events()

In [3]:
# Comenzaremos realizando un TF_idf de as busquedas
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [4]:
# Me quedo con los usuarios que tiene busquedas
modelos = events[events['model'].notnull()][['model','person']]

# Calculo de Tf_Idf
modelos_totales = modelos['model'].tolist()
IDF = vectorizer.fit_transform(modelos_totales)

# Asigno los TFIdf a cada busqueda
resultados = []
for x in range(len(modelos)):
    resultados.append(IDF[x].sum()) 

modelos['TfIdf'] = resultados
modelos_tfidf = modelos[['person','TfIdf']]

# Agrupo por persona y calculo el promedio de los TFIdf
modelos_por_usuario = modelos_tfidf.groupby('person')['TfIdf'].agg({'promedio de TFIdf modelo':'mean','Suma de TFidf modelo':sum,'Maximo de TFIdf modelo':max,'Minimo de TFIdf modelo':min}).reset_index()

In [5]:
modelos_por_usuario.fillna(0,inplace=True)
modelos_por_usuario.head()

,person,promedio de TFIdf modelo,Suma de TFidf modelo,Maximo de TFIdf modelo,Minimo de TFIdf modelo
0,0008ed71,1.617163,4.851488,1.859879,1.300103
1,00091926,1.490856,557.580165,2.209395,1.000000
2,00091a7a,1.219303,3.657909,1.357806,1.000000
3,000ba417,1.798317,287.730661,2.299973,1.000000
4,000c79fe,1.000000,4.000000,1.000000,1.000000


In [6]:
modelos_por_usuario.to_csv('features_modelo.csv',index=False)